In [15]:

wanted_topics = ['hiring','hiring decisions','human resources','HR','bias']
 



def get_sorted_dictionary(elements):
    
    '''
    
    how is the dictionary organised:
    
    key -> element in the list
    value-> number of times the key appears in the list
    
    '''
    
    dictionary = {}
    ret = {}
    
    dictionary = {element:elements.count(element) for element in elements}
            
    sorted_dict =  sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    return sorted_dict

def get_k_most_recurring_words(list,k):
    counter = 0
    
    list_of_words = []

    for entity in list:
        #print(f'{entity[0]} appears {entity[1]} times')
        counter += 1
        list_of_words.append(entity)
        if counter == k:
            return list_of_words

#checks the number of items wanted_topics and current_topics have in common
def get_number_same_topic(current_topics):
    
    if(len(current_topics) == 0):
        return 0;
    
    return len(set(current_topics) & set(wanted_topics))
    


In [ ]:
import os
import PyPDF2
import nltk
import spacy
from collections import Counter
import yake
import string
from nltk.corpus import stopwords
from pke.unsupervised import YAKE

def remove_stopwords(text,stopwords):
    new = []
    splitted_text = text.split()
    for word in splitted_text:
        if word not in stopwords or word.isdigit() or word == 'MIT':
            new.append(word)
            
    return  " ".join(new)

possible_pdfs = []

# Folder Path
path = "/Users/benedettatessa/Downloads/pdfs 2"
  
# Change the directory
os.chdir(path)
  
# iterate through all files
for file in os.listdir():
    if file.endswith('.pdf'):
        pdf_path = f"{path}/{file}"
        #print(pdf_path)
        
        pdfFileObject =  open(pdf_path, 'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFileObject)
        text=''
        
        #get text from all pages
        for i in range(0,pdfReader.numPages):
            # creating a page object
            pageObj = pdfReader.getPage(i)
            # extracting text from page
            text=text+pageObj.extractText()


        text = text.translate(str.maketrans('', '', string.punctuation))
        stop = stopwords.words('english')
        text = remove_stopwords(text,stop)
        
        extractor = YAKE()
        
        extractor.load_document(input=text.lower(),
                        language='en',
                        normalization=None)
        
        
        extractor.candidate_selection(n=2)

        # 4. Calculate scores for the candidate keywords
        extractor.candidate_weighting(window=2,
                                      use_stems=True)

        # 5. Select 10 highest ranked keywords
        # Remove redundant keywords with similarity above 80%
        key_phrases = extractor.get_n_best(n=10, threshold=0.8)
        print(key_phrases)
        

        '''
        nlp = spacy.load("en_core_web_md")
        stopwords = nlp.Defaults.stop_words

        doc = nlp(text)
        
        tokens = []
        tokens_dict = {}
        
        # Iterate over the tokens 
        for token in doc:
            #include token in our analysis if not a punctuation, not a stopword, not a number
            if((not token.is_punct) and (not token.lemma_ in stopwords) and (not token.text.isnumeric())):
                tokens.append(token.lemma_ )
                
        tokens_dict = get_sorted_dictionary(tokens)
        recurring = get_k_most_recurring_words(tokens_dict,5)
        
        common = get_number_same_topic(recurring)
        
        if not (common == None) and common > 2 : 
            possible_pdfs.append(file)
            print(f'{file} requires a further ananlysis')  
        else:
            print('useless')
        '''
        

print('finished')
